In [42]:
from datetime import datetime, timedelta
import numpy as py
import pandas as pd
from sklearn import linear_model
import os
import psycopg2
from functions import *
from databasecon import *

from flask import Flask
from flask import request
app = Flask(__name__)

In [ ]:
@app.get('/predict-metrics')
def predict_metrics():
    assetId = request.args.get('assetId', default = 0, type = int)
    metricType = request.args.get('metricType', default = 0, type = int)
    
    df = pd.DataFrame(getMetrics(assetId, metricType))
    df = df.drop([0, 3, 4], axis="columns")
    
    df[2] = df[2].apply(str)
    df[2].apply(type)
    
    df[2] = df[2].apply(timeStringToMicroSeconds)
    df[1] = df[1].apply(float)
    
    values = df[1]
    dates = df.drop(1, axis="columns")
    
    lr = linear_model.LinearRegression()
    lr.fit(dates, values)
    
    date = "2022-08-28 18:46:47.857"
    date2 = "2022-09-30 18:46:47.857"
    dateMS = timeStringToMicroSeconds(date)
    dateMS2 = timeStringToMicroSeconds(date2)
    result = lr.predict([[dateMS],[dateMS2]])
    
    print('Results', result)
    
    return {"data":list(result)}

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2022 11:54:45] "GET /predict-metrics HTTP/1.1" 200 -


[208.53141416 173.55002745]


127.0.0.1 - - [03/Sep/2022 11:54:52] "GET /predict-metrics HTTP/1.1" 200 -


[208.53141416 173.55002745]


In [43]:
df = pd.DataFrame(getMetrics())
df

,0,1,2,3,4
0,3,100.0,2022-08-29 19:20:31.743,1,1
1,4,100.0,2022-09-29 19:20:31.743,1,1
2,5,100.0,2022-10-29 19:20:31.743,1,1
3,6,120.0,2022-12-29 19:20:31.743,1,1
4,2,100.0,2022-08-28 18:46:47.857,1,1
5,7,500.0,2022-08-28 18:46:47.857,1,2


In [44]:
df = df.drop([0, 3, 4], axis="columns")
df

,1,2
0,100.0,2022-08-29 19:20:31.743
1,100.0,2022-09-29 19:20:31.743
2,100.0,2022-10-29 19:20:31.743
3,120.0,2022-12-29 19:20:31.743
4,100.0,2022-08-28 18:46:47.857
5,500.0,2022-08-28 18:46:47.857


In [45]:
df[2] = df[2].apply(str)
df[2].apply(type)

0    <class 'str'>
1    <class 'str'>
2    <class 'str'>
3    <class 'str'>
4    <class 'str'>
5    <class 'str'>
Name: 2, dtype: object

In [46]:
df[2] = df[2].apply(timeStringToMicroSeconds)
df[1] = df[1].apply(float)
df

,1,2
0,100.0,1.661801e+15
1,100.0,1.664479e+15
2,100.0,1.667071e+15
3,120.0,1.672342e+15
4,100.0,1.661712e+15
5,500.0,1.661712e+15


In [57]:
values = df[1]
dates = df.drop(1, axis="columns")

dates

,2
0,1.661801e+15
1,1.664479e+15
2,1.667071e+15
3,1.672342e+15
4,1.661712e+15
5,1.661712e+15


In [49]:
def convertMicroSecondsToTime2(time):
    s = float(time/10**6)
    return datetime.fromtimestamp(s) - timedelta(days=1)

dates[2].apply(convertMicroSecondsToTime2)

0   2022-08-29 00:50:31.743
1   2022-09-29 00:50:31.743
2   2022-10-29 00:50:31.743
3   2022-12-29 00:50:31.743
4   2022-08-28 00:16:47.857
5   2022-08-28 00:16:47.857
Name: 2, dtype: datetime64[ns]

In [7]:
lr = linear_model.LinearRegression()
lr.fit(dates, values)

LinearRegression()

In [8]:
date = "2022-08-28 18:46:47.857"
date2 = "2022-09-30 18:46:47.857"
dateMS = timeStringToMicroSeconds(date)
dateMS2 = timeStringToMicroSeconds(date2)
dateMS
result = lr.predict([[dateMS],[dateMS2]])

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000
Press CTRL+C to quit
[2022-09-03 11:51:50,141] ERROR in app: Exception on /predict-metrics [GET]
Traceback (most recent call last):
  File "C:\Users\Macbook\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Macbook\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    return self.finalize_request(rv)
  File "C:\Users\Macbook\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1842, in finalize_request
    response = self.make_response(rv)
  File "C:\Users\Macbook\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2153, in make_response
    rv = self.json.response(rv)
  File "C:\Users\Macbook\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\json\provider.py", line 309, in response
    f"{self.dumps(obj, **dump_args)}\n", 